In [8]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pickle import load
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

from descartes import PolygonPatch

import helper_functions.config as config
import helper_functions.data_functions as data_functions
import helper_functions.plotting_functions as plotting_functions

def warn(*args, **kwargs):
    pass

import warnings
warnings.warn = warn

In [11]:
raw_data = pd.read_parquet(data_file)
data = raw_data[features]

data.rename(columns={
    "mean_air_2m": "temp", 
    "mean_apcp": "rain",
    "mean_rhum_2m": "humidity",
    "mean_dpt_2m": "dew_point",
    "mean_pres_sfc": "pressure",
    "mean_uwnd_10m": "uwind",
    "mean_vwnd_10m": "vwind",
    "mean_cloud_cover": "cloud_cover"
    
}, inplace=True)

prediction_data = data.drop('ignition', axis=1)

with open(lat_lon_bins_file, 'r') as lat_lon_bins_file:
    rows = csv.reader(lat_lon_bins_file)
    lat_lon_bins = list(rows)
    
date_list = pd.DatetimeIndex(prediction_data['date'].unique())

california = data_functions.get_california_polygon(states_shapefile)
multipoly = california.loc[16, 'geometry']
california_land_mass = multipoly[-1]

In [12]:
prediction_data = scale_features(prediction_data, features_to_scale, quantile_transformer, min_max_scaler)
prediction_data = onehot_month(prediction_data)
prediction_data = format_data(prediction_data, shape_parameters)
predictions = predict(prediction_data, hyperparameters, weights_file)
formatted_predictions = format_for_api(predictions, lat_lon_bins, date_list)

input_data is <class 'list'>, length: 410, member shape: (8760, 5, 22).


In [ ]:
left  = 0.125  # the left side of the subplots of the figure
right = 0.65   # the right side of the subplots of the figure
bottom = 0.1   # the bottom of the subplots of the figure
top = 0.9      # the top of the subplots of the figure
wspace = 0.2   # the amount of width reserved for blank space between subplots
hspace = 0.3   # the amount of height reserved for white space between subplots

fig_rows = 8
fig_cols = 2
plot_height = 65
plot_width = 30

plot_locations = [
    (0,0),(0,1),
    (1,0),(1,1),
    (2,0),(2,1),
    (3,0),(3,1),
    (4,0),(4,1),
    (5,0),(5,1),
    (6,0),(6,1),
    (7,0),(7,1)
]

num_contour_levels = 100

formatted_predictions['lat'] = formatted_predictions['lat'].astype('float32')
formatted_predictions['lon'] = formatted_predictions['lon'].astype('float32')

prediction_data_by_day = formatted_predictions.groupby('day')

fig, ax = plt.subplots(fig_rows, fig_cols, figsize=(plot_width, plot_height))
fig.subplots_adjust(left=left, bottom=bottom, right=right, top=top, wspace=wspace, hspace=hspace)

i = 0

while i < 16:
    day = str(date_list[i].date())
    predictions = prediction_data_by_day.get_group(day)
    predictions.drop('day', axis=1, inplace=True)

    xi, yi, zi = data_functions.regularize_grid(predictions, 'ignition_risk', 0.001)

    ax[plot_locations[i]].add_patch(PolygonPatch(california_land_mass, fc='none', ec='black', lw='2', zorder=2))
    ax[plot_locations[i]].contourf(xi, yi, zi, num_contour_levels, cmap='viridis')
    ax[plot_locations[i]].set_title(day, fontsize=18)
    ax[plot_locations[i]].set_aspect('equal', adjustable='box')
    
    i += 1

In [ ]:
predictions_file = '../deployment/data/predictions/2020-06-21_formatted_predictions.parquet'

In [ ]:
prediction_data = pd.read_parquet(predictions_file)
prediction_data['lat'] = prediction_data['lat'].astype('float32')
prediction_data['lon'] = prediction_data['lon'].astype('float32')

In [ ]:
left  = 0.125  # the left side of the subplots of the figure
right = 0.65   # the right side of the subplots of the figure
bottom = 0.1   # the bottom of the subplots of the figure
top = 0.9      # the top of the subplots of the figure
wspace = 0.2   # the amount of width reserved for blank space between subplots
hspace = 0.3   # the amount of height reserved for white space between subplots

fig_rows = 4
fig_cols = 2
plot_height = 35
plot_width = 30

plot_locations = [
    (0,0),(0,1),
    (1,0),(1,1),
    (2,0),(2,1),
    (3,0),(3,1)
]

num_contour_levels = 5

prediction_data_by_day = prediction_data.groupby('day')

fig, ax = plt.subplots(fig_rows, fig_cols, figsize=(plot_width, plot_height))
fig.subplots_adjust(left=left, bottom=bottom, right=right, top=top, wspace=wspace, hspace=hspace)

i = 0

for group_name, predictions in prediction_data_by_day:
    day = predictions['day'][0]
    predictions.drop('day', axis=1, inplace=True)

    xi, yi, zi = data_functions.regularize_grid(predictions, 'ignition_risk', 0.01)

    ax[plot_locations[i]].add_patch(PolygonPatch(california_land_mass, fc='none', ec='black', lw='2', zorder=2))
    ax[plot_locations[i]].contourf(xi, yi, zi, num_contour_levels, cmap='viridis')
    ax[plot_locations[i]].set_title(day, fontsize=18)
    ax[plot_locations[i]].set_aspect('equal', adjustable='box')
    
    i += 1